In [1]:
import numpy as np
from tifffile import imsave
from NtruEncrypt import *
from Polynomial import Zx
from num_to_polynomial import *

In [2]:
def scale_array(dat, out_range=(0, 1)):
    domain = [np.min(dat, axis=0), np.max(dat, axis=0)]

    def interp(x):
        return out_range[0] * (1.0 - x) + out_range[1] * x

    def uninterp(x):
        b = 0
        if (domain[1] - domain[0]) != 0:
            b = domain[1] - domain[0]
        else:
            b =  1.0 / domain[1]
        return (x - domain[0]) / b

    return interp(uninterp(dat))


In [3]:
def revert_dimension(lst,a,b):
    rd = []
    for i in range(a):
        t = []
        for j in range(b):
            t.append(lst[j+i*b])
        rd.append(t)
    return rd


In [4]:
def flatten_list(lst):
    return [val for l in lst for val in l]

In [5]:
d = 5
p = 3
q = 128

message = input("Enter Message: ")

print('Curve Parameters')
elliptic_a = int(input("Enter A: "))
elliptic_b = int(input("Enter B: "))

character_polynomials,N = koblitz_encoder(message,elliptic_a,elliptic_b) 

public_key,private_key = generate_keypair(p,q,d,N)

print('\nPublic Key = ',end='')
print(public_key.print_polynomial())

print('\nEncrypted = ')
cipher_polys = []
coefficients = []
for element in character_polynomials:
    cipher_text = encrypt(element,public_key,d,N,q)
    cipher_text.coeffs = list(map(int, cipher_text.coeffs))
    cipher_polys.append(cipher_text)
    coefficients.append(cipher_text.coeffs)
print(coefficients)

Enter Message: Unnikrishnan
Curve Parameters
Enter A: 9
Enter B: 7

Public Key = (-15x^14) + (-61x^13) + (38x^12) + (11x^11) + (-39x^10) + (-42x^9) + (-41x^8) + (-18x^7) + (-40x^6) + (-5x^5) + (-20x^4) + (-15x^3) + (50x^2) + (1x) + (53)

Encrypted = 
[[38, -18, 36, -26, -20, 20, -51, -46, 33, -32, 39, 40, -43, 35, 14], [12, -18, 32, -22, 27, -5, -1, -54, -28, -32, -46, 56, 53, -50, 60], [-28, -12, 12, -12, 0, 57, -20, 20, 36, -9, -1, -61, -43, 7, -30], [-25, -56, 19, 51, 8, -17, -58, 13, 58, 12, -21, -54, 14, 56, 13], [63, -56, -10, 63, -20, 30, 57, -45, -17, 31, 24, -52, 47, -19, 46], [27, -12, -51, 37, 47, -38, 30, 10, -60, -60, -8, -25, -38, -47, 46], [-20, -32, -36, -20, 30, 6, 61, -46, -10, -45, 3, 4, 39, 57, 52], [-32, 27, -19, 63, 4, -62, 9, -51, -17, 61, 46, 35, 39, -27, -59], [43, -61, -36, 18, 45, -63, -14, -46, -37, 8, -9, -40, 59, 61, 56], [-24, 10, -56, -49, 21, -36, 33, 32, -54, 35, 4, 31, -21, 27, -29], [13, 45, -32, -13, 21, 21, -25, 2, 62, 49, 38, 10, -45, 6, -64], [-4

In [6]:
x_trans = scale_array(flatten_list(coefficients))

print('Convert this to image:')
to_img = revert_dimension(x_trans,len(coefficients),N)
print(to_img)

to_img = np.asarray(to_img)

print('Generating Image...')

imsave('encrypted_image.tif', to_img)

Convert this to image:
[[0.8031496062992126, 0.36220472440944884, 0.7874015748031497, 0.2992125984251969, 0.3464566929133858, 0.6614173228346457, 0.10236220472440945, 0.14173228346456693, 0.7637795275590551, 0.25196850393700787, 0.8110236220472441, 0.8188976377952756, 0.16535433070866143, 0.7795275590551181, 0.6141732283464567], [0.5984251968503937, 0.36220472440944884, 0.7559055118110236, 0.33070866141732286, 0.7165354330708661, 0.4645669291338583, 0.49606299212598426, 0.07874015748031496, 0.28346456692913385, 0.25196850393700787, 0.14173228346456693, 0.9448818897637795, 0.9212598425196851, 0.11023622047244094, 0.9763779527559056], [0.28346456692913385, 0.4094488188976378, 0.5984251968503937, 0.4094488188976378, 0.5039370078740157, 0.952755905511811, 0.3464566929133858, 0.6614173228346457, 0.7874015748031497, 0.4330708661417323, 0.49606299212598426, 0.023622047244094488, 0.16535433070866143, 0.5590551181102362, 0.2677165354330709], [0.30708661417322836, 0.06299212598425197, 0.65354330

In [7]:
import imageio
from numpy import asarray
from PIL import Image

image = imageio.imread('encrypted_image.tif')

pixels = asarray(image)

print('Data Type: %s' % pixels.dtype)
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
# convert from integers to floats
pixels = pixels.astype('float64')

print(pixels)
decoded_coefficients = []
for p in pixels:
    for j in p:
        decoded_coefficients.append(j)



Data Type: float64
Min: 0.000, Max: 1.000
[[0.80314961 0.36220472 0.78740157 0.2992126  0.34645669 0.66141732
  0.1023622  0.14173228 0.76377953 0.2519685  0.81102362 0.81889764
  0.16535433 0.77952756 0.61417323]
 [0.5984252  0.36220472 0.75590551 0.33070866 0.71653543 0.46456693
  0.49606299 0.07874016 0.28346457 0.2519685  0.14173228 0.94488189
  0.92125984 0.11023622 0.97637795]
 [0.28346457 0.40944882 0.5984252  0.40944882 0.50393701 0.95275591
  0.34645669 0.66141732 0.78740157 0.43307087 0.49606299 0.02362205
  0.16535433 0.55905512 0.26771654]
 [0.30708661 0.06299213 0.65354331 0.90551181 0.56692913 0.37007874
  0.04724409 0.60629921 0.96062992 0.5984252  0.33858268 0.07874016
  0.61417323 0.94488189 0.60629921]
 [1.         0.06299213 0.42519685 1.         0.34645669 0.74015748
  0.95275591 0.1496063  0.37007874 0.7480315  0.69291339 0.09448819
  0.87401575 0.35433071 0.86614173]
 [0.71653543 0.40944882 0.1023622  0.79527559 0.87401575 0.20472441
  0.74015748 0.58267717 0.0314

In [8]:
x_inv = scale_array(decoded_coefficients,(np.min(flatten_list(coefficients), axis=0), np.max(flatten_list(coefficients), axis=0)))
x_inv = [round(i) for i in x_inv]
x_inv = list(map(int, x_inv))
decoded_coefficients = revert_dimension(x_inv,len(coefficients),N)

In [9]:
print(decoded_coefficients)

[[38, -18, 36, -26, -20, 20, -51, -46, 33, -32, 39, 40, -43, 35, 14], [12, -18, 32, -22, 27, -5, -1, -54, -28, -32, -46, 56, 53, -50, 60], [-28, -12, 12, -12, 0, 57, -20, 20, 36, -9, -1, -61, -43, 7, -30], [-25, -56, 19, 51, 8, -17, -58, 13, 58, 12, -21, -54, 14, 56, 13], [63, -56, -10, 63, -20, 30, 57, -45, -17, 31, 24, -52, 47, -19, 46], [27, -12, -51, 37, 47, -38, 30, 10, -60, -60, -8, -25, -38, -47, 46], [-20, -32, -36, -20, 30, 6, 61, -46, -10, -45, 3, 4, 39, 57, 52], [-32, 27, -19, 63, 4, -62, 9, -51, -17, 61, 46, 35, 39, -27, -59], [43, -61, -36, 18, 45, -63, -14, -46, -37, 8, -9, -40, 59, 61, 56], [-24, 10, -56, -49, 21, -36, 33, 32, -54, 35, 4, 31, -21, 27, -29], [13, 45, -32, -13, 21, 21, -25, 2, 62, 49, 38, 10, -45, 6, -64], [-4, 13, 12, -9, 2, 38, -42, -45, 10, -20, -63, -26, -55, -42, -11]]


In [10]:
print(coefficients)

[[38, -18, 36, -26, -20, 20, -51, -46, 33, -32, 39, 40, -43, 35, 14], [12, -18, 32, -22, 27, -5, -1, -54, -28, -32, -46, 56, 53, -50, 60], [-28, -12, 12, -12, 0, 57, -20, 20, 36, -9, -1, -61, -43, 7, -30], [-25, -56, 19, 51, 8, -17, -58, 13, 58, 12, -21, -54, 14, 56, 13], [63, -56, -10, 63, -20, 30, 57, -45, -17, 31, 24, -52, 47, -19, 46], [27, -12, -51, 37, 47, -38, 30, 10, -60, -60, -8, -25, -38, -47, 46], [-20, -32, -36, -20, 30, 6, 61, -46, -10, -45, 3, 4, 39, 57, 52], [-32, 27, -19, 63, 4, -62, 9, -51, -17, 61, 46, 35, 39, -27, -59], [43, -61, -36, 18, 45, -63, -14, -46, -37, 8, -9, -40, 59, 61, 56], [-24, 10, -56, -49, 21, -36, 33, 32, -54, 35, 4, 31, -21, 27, -29], [13, 45, -32, -13, 21, 21, -25, 2, 62, 49, 38, 10, -45, 6, -64], [-4, 13, 12, -9, 2, 38, -42, -45, 10, -20, -63, -26, -55, -42, -11]]


In [11]:
if coefficients == decoded_coefficients:
    print('Complete Match!')
else:
    print('Not Equal!')

Complete Match!
